In [10]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import sys 
import matplotlib 
import os
%matplotlib inline

In [8]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 2.7.13 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:17:26) [MSC v.1500 64 bit (AMD64)]
Pandas version 0.20.1
Matplotlib version 2.0.2


In [36]:
pd.read_json?
# path
jsonloc = r'C:\Users\Sophia\DTI_semifinal\coarse-discourse\coarse_discourse_dataset.json'

# take a look at the json annotation file
df = pd.read_json(jsonloc, lines = True)
df.dtypes
print df.head()
df.columns



   is_self_post                                              posts  \
0           1.0  [{u'is_first_post': True, u'majority_link': u'...   
1           1.0  [{u'is_first_post': True, u'majority_link': u'...   
2           1.0  [{u'is_first_post': True, u'majority_link': u'...   
3           1.0  [{u'is_first_post': True, u'majority_link': u'...   
4           1.0  [{u'is_first_post': True, u'majority_link': u'...   

          subreddit                                    title  \
0  100movies365days                  DTX120: #87 - Nashville   
1           100sets  Male, 23 years old. Going for 100 sets!   
2      1200isplenty         122cal black currant cheesecake!   
3              1911         Need help finding a Springfield!   
4              1911              Help with a possible trade?   

                                                 url  
0  https://www.reddit.com/r/100movies365days/comm...  
1  https://www.reddit.com/r/100sets/comments/omv7...  
2  https://www.reddit.com/r/1

Index([u'is_self_post', u'posts', u'subreddit', u'title', u'url'], dtype='object')

In [3]:
'''Joins Coarse-Discourse annotations with Reddit data via Reddit API.'''
# Copyright 2017 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import __future__
from __future__ import print_function

import json
import os
import time

import praw

'''
Requirements: PRAW python library - https://praw.readthedocs.io/en/latest/index.html
              current version: 4.4.0

Date: 04/02/2017

This script adds fields to the coarse-discourse dataset for each comment including the 
text of the comment and the author. The information is taken from the Reddit API.

You can augment this script to gather other information about the comment such as upvote 
or downvote count or about the comment author, such as their karma, from the fields
that the Reddit API provides.


Note:

There may be discrepancies due to changes made between when the coarse-discourse
data was collected (August 2016) and when you are accessing the API, such as missing 
comments or comment text.

You should be able to overcome discrepancies by downloading the full Reddit dump from 
the beginning up until 09/2016 found in various places such as: 

https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/
https://archive.org/details/2015_reddit_comments_corpus
https://bigquery.cloud.google.com/dataset/fh-bigquery:reddit_posts

and using that data instead of the Reddit API to collect the comment texts and author names.
'''

# Replace below with information provided to you by Reddit when registering your script
reddit = praw.Reddit(client_id="QutcH0SZqEwjBw",
                     client_secret="BDIt_Tif5Eo5asnYBC-lZdQ8Q5E",
                     user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                     AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36")

with open('coarse_discourse_dataset.json') as jsonfile:
    lines = jsonfile.readlines()
    dump_with_reddit = open('coarse_discourse_dump_reddit.json', 'w')

    for line in lines:
        reader = json.loads(line)
        print(reader['url'])

        submission = reddit.submission(url=reader['url'])

        # Annotators only annotated the 40 "best" comments determined by Reddit
        submission.comment_sort = 'best'
        submission.comment_limit = 40

        post_id_dict = {}

        for post in reader['posts']:
            post_id_dict[post['id']] = post

        try:
            full_submission_id = 't3_' + submission.id
            if full_submission_id in post_id_dict:
                post_id_dict[full_submission_id]['body'] = submission.selftext

                # For a self-post, this URL will be the same URL as the thread.
                # For a link-post, this URL will be the link that the link-post is linking to.
                post_id_dict[full_submission_id]['url'] = submission.url
                if submission.author:
                    post_id_dict[full_submission_id]['author'] = submission.author.name

            submission.comments.replace_more(limit=0)
            for comment in submission.comments.list():
                full_comment_id = 't1_' + comment.id
                if full_comment_id in post_id_dict:
                    post_id_dict[full_comment_id]['body'] = comment.body
                    if comment.author:
                        post_id_dict[full_comment_id]['author'] = comment.author.name

        except Exception as e:
            print('Error %s' % (e))

        found_count = 0
        for post in reader['posts']:
            if not post.has_key('body'):
                print("Can't find %s in URL: %s" % (post['id'], reader['url']))
            else:
                found_count += 1

        print('Found %s posts out of %s' % (found_count, len(reader['posts'])))

        dump_with_reddit.write(json.dumps(reader) + '\n')

        # To keep within Reddit API limits
        time.sleep(2)


https://www.reddit.com/r/100movies365days/comments/1bx6qw/dtx120_87_nashville/
Found 4 posts out of 4
https://www.reddit.com/r/100sets/comments/omv7p/male_23_years_old_going_for_100_sets/
Found 7 posts out of 7
https://www.reddit.com/r/1200isplenty/comments/259tbh/122cal_black_currant_cheesecake/
Found 8 posts out of 8
https://www.reddit.com/r/1911/comments/16h61h/need_help_finding_a_springfield/
Found 19 posts out of 19
https://www.reddit.com/r/1911/comments/35igzp/help_with_a_possible_trade/
Found 8 posts out of 8
https://www.reddit.com/r/2007scape/comments/195abs/good_youtubers_to_watch_for_2007/
Found 6 posts out of 6
https://www.reddit.com/r/2007scape/comments/1uz5j7/training_or_slayer/
Found 18 posts out of 18
https://www.reddit.com/r/2007scape/comments/20pv9h/never_give_up/
Found 7 posts out of 7
https://www.reddit.com/r/2007scape/comments/25hor7/new_skill/
Found 5 posts out of 5
https://www.reddit.com/r/2007scape/comments/27aan0/recording_osrs_with_frapsalternative_need_help/
F

https://www.reddit.com/r/ARKone/comments/4hnviz/happy_multiplier_for_xp/
Found 5 posts out of 5
https://www.reddit.com/r/ASU/comments/2xj6lw/does_asu_have_a_sound_room_to_record_things_in/
Found 11 posts out of 11
https://www.reddit.com/r/Aberdeen/comments/20btim/activitiesevents_to_meet_new_people/
Found 7 posts out of 7
https://www.reddit.com/r/AcademicPsychology/comments/4d34kb/how_to_add_sshrc_to_a_cv/
Found 6 posts out of 6
https://www.reddit.com/r/Accounting/comments/2oduo3/anyone_have_a_decent_overall_gpa_and_lower_end/
Found 10 posts out of 10
https://www.reddit.com/r/Accounting/comments/339r8j/lets_post_starting_salary_offers_for_new_hires_of/
Found 38 posts out of 38
https://www.reddit.com/r/Accounting/comments/4ae3s8/undergrad_student_starting_to_feel_under/
Found 4 posts out of 4
https://www.reddit.com/r/Accounting/comments/n96zc/few_questions_from_actg_100/
Found 9 posts out of 9
https://www.reddit.com/r/ActionFigures/comments/43nuh6/sunnydale_library_or_xavirers_school_fo

Found 7 posts out of 7
https://www.reddit.com/r/Anxiety/comments/4eg41q/how_do_i_beat_my_anticipatory_and_flight_anxiety/
Found 5 posts out of 5
https://www.reddit.com/r/Anxiety/comments/4hl8l8/i_wrote_a_letter_to_myself_to_read_when_im/
Found 23 posts out of 23
https://www.reddit.com/r/Anxiety/comments/4idjm1/anxiety_and_needing_to_poop/
Found 7 posts out of 7
https://www.reddit.com/r/ApocalypseRising/comments/1z0882/whats_the_best_stranger_teamup_youve_had/
Found 16 posts out of 16
https://www.reddit.com/r/ApocalypseRising/comments/4e5uh5/can_i_be_banned_for_picking_up_a_hackers_items/
Found 15 posts out of 15
https://www.reddit.com/r/AppNana/comments/2p74wn/a598454/
Found 4 posts out of 4
https://www.reddit.com/r/AppleWatch/comments/3bbtec/this_may_not_be_the_appropriate_place_to_post/
Found 4 posts out of 4
https://www.reddit.com/r/ApplyingToCollege/comments/4jeje4/is_there_a_list_of_schools_that_look_at_intended/
Found 7 posts out of 7
https://www.reddit.com/r/Aquariums/comments/2

https://www.reddit.com/r/AskReddit/comments/12wms6/south_park_had_a_postelection_episode_air_on/
Found 15 posts out of 15
https://www.reddit.com/r/AskReddit/comments/12z5sm/if_you_could_forget_one_thing_from_your_life/
Found 8 posts out of 8
https://www.reddit.com/r/AskReddit/comments/12zxei/could_anybody_tell_me_anything_about_this_lighter/
Found 9 posts out of 9
https://www.reddit.com/r/AskReddit/comments/1392ed/waiters_of_reddit_do_you_feel_that_because/
Can't find t1_c71uxe4 in URL: https://www.reddit.com/r/AskReddit/comments/1392ed/waiters_of_reddit_do_you_feel_that_because/
Found 7 posts out of 8
https://www.reddit.com/r/AskReddit/comments/14dtff/what_pop_culture_expression_or_any_phrase_really/
Found 4 posts out of 4
https://www.reddit.com/r/AskReddit/comments/14hr56/how_do_babies_laughcryburp_in_movies_and_tv_shows/
Found 11 posts out of 11
https://www.reddit.com/r/AskReddit/comments/14n5ny/what_about_your_childhood_do_you_most_regret_if/
Found 15 posts out of 15
https://www.re

Found 39 posts out of 39
https://www.reddit.com/r/AskReddit/comments/1kpp36/if_you_tell_people_you_have_done_a_good_deed_does/
Found 13 posts out of 13
https://www.reddit.com/r/AskReddit/comments/1ltzjm/australia_is_losing_its_mojo_now_the_resource/
Found 25 posts out of 25
https://www.reddit.com/r/AskReddit/comments/1luu79/if_most_jobs_were_replaced_by_robots_what_work/
Found 13 posts out of 13
https://www.reddit.com/r/AskReddit/comments/1lz5zt/what_last_line_from_a_book_will_always_stick_with/
Found 9 posts out of 9
https://www.reddit.com/r/AskReddit/comments/1m9vgp/which_of_your_possessions_do_you_think_is/
Found 7 posts out of 7
https://www.reddit.com/r/AskReddit/comments/1mqbp2/what_songshowmovie_is_your_guilty_please/
Found 4 posts out of 4
https://www.reddit.com/r/AskReddit/comments/1mvwtc/whats_the_most_embarrassing_moment_youve_been/
Found 6 posts out of 6
https://www.reddit.com/r/AskReddit/comments/1n6xat/what_gets_you_up_in_the_morning/
Found 13 posts out of 13
https://www.r

...

Found 4 posts out of 4
https://www.reddit.com/r/whatsthatbook/comments/2zw9t2/horror_story_collection_that_i_recall_reading_at/
Found 6 posts out of 6
https://www.reddit.com/r/whatstheword/comments/1bak5q/wtw_that_means_embeddedness_ie_the_state_of_being/
Found 12 posts out of 12
https://www.reddit.com/r/whatstheword/comments/32da76/whats_the_word_for_the_mirroring_done_in_a_movie/
Found 7 posts out of 7
https://www.reddit.com/r/whatstheword/comments/389c3n/wtw_it_refers_to_when_an_older_piece_of_writing/
Can't find t1_crtbbub in URL: https://www.reddit.com/r/whatstheword/comments/389c3n/wtw_it_refers_to_when_an_older_piece_of_writing/
Found 14 posts out of 15
https://www.reddit.com/r/whatstheword/comments/3w001x/wtw_for_the_feeling_that_is_both_happy_and/
Found 17 posts out of 17
https://www.reddit.com/r/whatstheword/comments/4273dp/wtw_for_when_a_person_is_stuck_in_a_bad_situation/
Can't find t1_cz84hzy in URL: https://www.reddit.com/r/whatstheword/comments/4273dp/wtw_for_when_a_pers

Can't find t1_c6wrode in URL: https://www.reddit.com/r/wicked_edge/comments/12nn1k/helped_a_buddy_move_found_this_shelf_in_an_old/
Found 7 posts out of 8
https://www.reddit.com/r/wicked_edge/comments/1w1h88/uk_cheap_feather_blades_30_for_444_delivered/
Found 15 posts out of 15
https://www.reddit.com/r/wicked_edge/comments/25tpvd/after_a_month_i_have_found_my_perfect_shave/
Found 6 posts out of 6
https://www.reddit.com/r/wicked_edge/comments/2dmget/new_york_shaving_stores/
Found 6 posts out of 6
https://www.reddit.com/r/wicked_edge/comments/373dwq/next_set_of_stones/
Found 6 posts out of 6
https://www.reddit.com/r/wicked_edge/comments/4eeujc/first_streight_razor_shave/
Found 5 posts out of 5
https://www.reddit.com/r/wicked_edge/comments/sx1aw/brushes_not_holding_as_much_water_question/
Found 9 posts out of 9
https://www.reddit.com/r/wicked_edge/comments/z1wuj/shave_humor/
Found 11 posts out of 11
https://www.reddit.com/r/widowers/comments/4ezgds/opportunity_for_widows_and_widowers/
Foun

Found 5 posts out of 5
https://www.reddit.com/r/woodworking/comments/2m01e9/need_some_help_lacquer_acting_up/
Found 6 posts out of 6
https://www.reddit.com/r/woodworking/comments/2rp6o4/on_the_subject_of_learning_how_to_work_wood/
Found 6 posts out of 6
https://www.reddit.com/r/woodworking/comments/3btntx/question_about_router_tables/
Found 7 posts out of 7
https://www.reddit.com/r/woodworking/comments/3cuhqn/table_saw_or_router_table/
Found 16 posts out of 16
https://www.reddit.com/r/woodworking/comments/x8z5u/help_a_newb_understand_the_difference_between_a/
Found 9 posts out of 9
https://www.reddit.com/r/woodworking/comments/ysbng/i_would_like_to_build_a_simpler_version_of_this_i/
Found 9 posts out of 9
https://www.reddit.com/r/woodysgamertag/comments/1rv4ue/does_woody_not_play_console_anymore/
Found 7 posts out of 7
https://www.reddit.com/r/workaholics/comments/20csuw/gif_request_thread_s04e08_beer_heist/
Found 31 posts out of 31
https://www.reddit.com/r/worldbuilding/comments/1l6k7

https://www.reddit.com/r/xbox360/comments/zeteu/my_360_could_be_broken_any_help/
Found 18 posts out of 18
https://www.reddit.com/r/xboxone/comments/1kj859/do_you_think_there_would_be_a_market_for_modded/
Found 6 posts out of 6
https://www.reddit.com/r/xboxone/comments/1mzil7/question_about_xbox_live_purchasing_on_xbox_one360/
Found 6 posts out of 6
https://www.reddit.com/r/xboxone/comments/1qwmjp/what_do_you_guys_think_of_the_reviews/
Found 19 posts out of 19
https://www.reddit.com/r/xboxone/comments/1rj7sq/upload_studios/
Found 6 posts out of 6
https://www.reddit.com/r/xboxone/comments/1uti5w/xbox_one_connection_help/
Found 8 posts out of 8
https://www.reddit.com/r/xboxone/comments/1xiapt/turtle_beach_xo_four_and_seven_confirmed_release/
Found 41 posts out of 41
https://www.reddit.com/r/xboxone/comments/200gfq/request_the_ability_to_change_tv_inputsource/
Found 22 posts out of 22
https://www.reddit.com/r/xboxone/comments/209ghk/gamechat_audio_issues_controller_freezing_xbox/
Found 5 p

Found 10 posts out of 10
https://www.reddit.com/r/zen/comments/2rjq21/science_and_human_beings/
Found 41 posts out of 41
https://www.reddit.com/r/zen/comments/4bqqeo/everything_is_black_and_white/
Found 20 posts out of 20
https://www.reddit.com/r/zen/comments/4fikn0/the_single_spiritual_brilliance/
Found 15 posts out of 15
https://www.reddit.com/r/zenfone2/comments/4flbb6/my_screen_went_black/
Found 6 posts out of 6
https://www.reddit.com/r/zengmlol/comments/40sxge/lms_bug/
Found 4 posts out of 4
https://www.reddit.com/r/zizek/comments/2y3014/the_real_as_the_condition_of_ontological/
Found 3 posts out of 3
https://www.reddit.com/r/zizek/comments/472of1/antagonistic_struggle_vs_tolerance_as_distinct_as/
Found 5 posts out of 5
https://www.reddit.com/r/zombiespartyup/comments/1cfqdc/need_2_for_mob_of_the_dead_at_release/
Found 4 posts out of 4
https://www.reddit.com/r/zootopia/comments/48toyo/frozen_2/
Found 6 posts out of 6
